In [ ]:
import pygame
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import time
import scipy.signal
import random

# GAME OF LIFE 

Nous sommes ravis de vous présenter notre premier projet en Python.

Le but de ce projet était de construire un Jeu de La Vie ou "Game of Life" afin de mettre en pratique nos connaissances. 

Nous avons travaillé dur pendant un mois et demi en respectant un ***plan de travail structuré*** qui vous serra présenté ci-dessous. 

Dans un premier temps, il a fallut mettre en place une ***organisation***. Cela nous a permit de respecter les deadlines tout en travaillant sur le projet de façons ***constante***. 
Ainsi, nous avons privilégier une première réunion de ***préconception*** pour éviter de nous lancer tête baissée dans le projet. 
Au cours de cette préconception nous nous sommes reparti les tâches selon les envies et les facilités de chacun. 
Nous avons également mis en place un document partagé et un Discord pour suivre les avancées du projet en continu. 
Chacun était libre de poser des questions et nous sommes resté soudé pour permettre au projet de voir le jour. 
De plus, nous avons choisi de mettre en place un ***projet GitHub*** afin de mettre en commun tous les dossiers crée. Cela a permit à chacun de tester le code et de suivre son avancement. 

Après avoir mis en place notre plan de travail nous avons commencé le codage. 
Il était necessaire pour nous de commencer par créer le jeu en console avant de l implémenter sous forme graphique. 

## A. Moteur de Jeu: 

### La Grille :
Pour cela on a créer une ***classe Grille()*** contenant des méthodes permettant son actualisation. 
    1- On doit définir pour chaque cellule de la grille ses cellules voisines et leur état = morte ou vivante. 
    --> ici une cellule correspondra à une case de la grille elle est donc définit par ***self.grille2D[lignes][colonnes]*** 

In [ ]:
def find_voisin(self): 
    list_voisins = []

    for lignes in range(-1, 2): 
        for colonnes in range(-1,2): 
            voisin_ligne = self.grille2D[lignes] + lignes
            voisin_colonne = self.grille2D[colonnes] + colonnes 

            voisin_OK = True 
            if (voisin_ligne) == self.grille2D[lignes] and (voisin_colonne) == self.grille2D[colonnes]: 
                voisin_OK = False
            if (voisin_ligne) < 0 or (voisin_colonne)>= self.colonnes: 
                voisin_OK = False
            if (voisin_colonne) < 0 or (voisin_ligne) >= self.lignes: 
                voisin_OK = False 
        
            if voisin_OK : 
                list_voisins.append(self.grille2D[voisin_ligne][voisin_colonne])
    return list_voisins  

2 - Après avoir determiner le nombre de cellules voisines de chaque cellule en stockant leur état dans 2 listes distinctes on va finalement faire évoluer la grille en fonction des 3 règles du jeu. 
  - on créer de 2 listes pour stocker les cellules vivantes et mortes
  - on parcours la grille en modifiant l'état des cellulles en fonction des 3 règles 

In [ ]:
def update_board(self): 

        list_vivantes = []
        list_mortes = []

        for lignes in range (len(self.grille2D)): 
            for colonnes in range(len(self.grille2D[lignes])): 
                find_voisin = self.find_voisin(lignes, colonnes)
                nb_voisin_alive = []
                
                for cell_voisine in find_voisin : 
                    if cell_voisine.check_actual_state(): 
                        nb_voisin_alive.append(cell_voisine)

                cell = self.grille2D[lignes][colonnes]
                etat_cell = cell.check_actual_state()

                if etat_cell == True : 
                    if len(nb_voisin_alive) == 2 or len(nb_voisin_alive) == 3: 
                        list_vivantes.append(cell)
                    if len(nb_voisin_alive) < 2 or len(nb_voisin_alive) > 3:
                        list_mortes.append(cell)
                else: 
                    if len(nb_voisin_alive) == 3: 
                        list_vivantes.append(cell)
        for cellule in list_vivantes: 
            cellule.state == 1
        for cellule in list_mortes:
            cellule.state == 0 

### Initialiser une grille aléatoirement

In [ ]:
nb_colonnes = 50
nb_lignes = 50

grille2D = [[0 for _ in range(nb_colonnes)] for _ in range(nb_lignes)]

def build_grille(grille2D):
    for ligne in grille2D:
        for colonne in ligne:
            alea = random.randint(0, 2)
            if alea == 1:
                grille2D[ligne][colonne] = 1

Une fois la grille fonctionnelle, nous allons maintenant nous concentrer sur l'affichage de notre jeu en utilisant les fonctions aupréalablement créer. 

## B. Partie interface

***ATTENTION !*** 
Pour éviter un problème ou un bug nous devons ABSOLUEMENT vérifier que la taille entrée est bien un entier réel positif. 
Nous avons donc utilisé la ***méthode du blindage***. 

Il faudra aussi caster le nombre en entier car tout ce qui est entrée au clavier est forcément considéré par défaut comme un string (chaîne de caractère). 

In [ ]:
def draw_board(self):
        self.screen.fill((135, 206, 250))
        
        # Dessin des celulles
        for i in range(self.game.height):
            for j in range(self.game.width):
                if self.game.board[i, j] == 1:
                    pygame.draw.rect(
                        self.screen,
                        (0, 0, 100),
                        (
                            j * self.cell_size + 40,
                            i * self.cell_size + 40,
                            self.cell_size,
                            self.cell_size,
                        ),
                    )

        # Dessin du cadrillage
        for i in range(self.game.height + 1):
            pygame.draw.line(self.screen,(0, 0, 0),(40, i * self.cell_size + 40),(40 + self.game.width * self.cell_size, i * self.cell_size + 40),1)

        for j in range(self.game.width + 1):
            pygame.draw.line(self.screen,(0, 0, 0),(j * self.cell_size + 40, 40),(j * self.cell_size + 40, 40 + self.game.height * self.cell_size),1)


In [ ]:
def update_cell_on_click(self, pos):
    i, j = (pos[1] - 40) // self.cell_size, (pos[0] - 40) // self.cell_size

    if (
        0 <= i < self.game.height
        and 0 <= j < self.game.width
        and not self.placement
    ):
        self.game.board[i, j] = 1 - self.game.board[i, j]
        self.draw_board()   

In [ ]:
def transition_between_state(self):
        if self.update:
                self.game.update_board()
                self.draw_board()
                self.draw_button()

In [ ]:
# Création d'un évènement de clock
CLOCK_EVENT = pygame.USEREVENT + 1
pygame.time.set_timer(CLOCK_EVENT, 300)

def handle_events(self):
    for event in pygame.event.get():
        if event.type == pygame.USEREVENT + 1:
            if self.pause == False:
                self.update = True
                self.transition_between_state()

In [ ]:
# positionnement structures complexes

## C. Sauvegarde et Chargement

In [ ]:
def save_to_file(self, filename):
    np.savetxt(filename, self.board, fmt="%d")

def load_from_file(self, filename):
    self.board = np.loadtxt(filename, dtype=int)

## D. Partie Optimisation

In [ ]:
def update_board(self):
    kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])
    neighbors_count = scipy.signal.convolve2d(
        self.board, kernel, mode="same", boundary="fill", fillvalue=0
    )
    self.board = np.where(
        (self.board == 1) & ((neighbors_count < 2) | (neighbors_count > 3)),
        0,
        np.where(
            (self.board == 1) & ((neighbors_count == 2) | (neighbors_count == 3)),
            1,
            np.where((self.board == 0) & (neighbors_count == 3), 1, self.board),
        ),
    )

In [ ]:
def transition_between_state(self):
    start_time = time.time()
    self.game.update_board()
    end_time = time.time()
    elapsed_time = (end_time - start_time) * 1000
    self.elapsed_time.append(elapsed_time)

### Tests du temps d'exécution pour une matrice de 100x100 générée aléatoirement 100 fois.

résultats : 1ms

In [80]:
import time
import numpy as np
import scipy.signal

def update_board(board):
    kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])
    neighbors_count = scipy.signal.convolve2d(
        board, kernel, mode="same", boundary="fill", fillvalue=0
    )

    condition_1 = (board == 1) & ((neighbors_count < 2) | (neighbors_count > 3))
    condition_2 = (board == 1) & ((neighbors_count == 2) | (neighbors_count == 3))
    condition_3 = (board == 0) & (neighbors_count == 3)

    board = np.where(condition_1, 0, np.where(condition_2, 1, np.where(condition_3, 1, board)))



start_time = time.time()
for i in range(100):
    board = np.random.choice([0, 1], size=(100, 100), p=[0.5, 0.5])
    update_board(board)

end_time = time.time()
elapsed_time = (end_time - start_time) * 1000 / 100
print(f"---- Temps moyen d'exécution {elapsed_time:.2f} ms ----")

---- Temps moyen d'exécution 0.95 ms ----


## E. Analyse de données et présentation

De plus, nous souhaitons définir et connaitre leur état à chaque instant. Cela nous permettra de générer des courbes et facilitera l évolution de la grille. 
On aura donc 3 méthodes : 


### La Grille : 
Nous avons opté pour une classe Grille () qui permettra de générer une grille avec des paramètres génériques tels que sa largeur et sa longueur (nombre de lignes / nombre de colonnes)
Il nous a paru également plus efficace d utiliser des méthodes pour créer, remplir, afficher et mettre à jour une grille de taille NxN. 
Ainsi, nous avons la définition de notre classe comme suit :  

### Evolution du jeu :
Pour se faire, les 3 règles suivantes doivent être respectées : 
- une cellule vivante survit si elle a 2 ou 3 voisins vivants.
- une cellule vivante meurt de solitude ou de surpopulation.
- une cellule morte peut devenir vivante si elle a exactement 3 voisins vivants.

Nous avons crée 2 méthodes pour respecter ces règles. 

#### Définir les voisins de chaque cellule dans la grille :  
* on crée un liste de cellules qui contiendra les cellules voisines et qui sera stocké dans les paramètres de la cellule. 
* on parcours chaque cellule de la grille en partant de la cellule à tester. 
* on parcours la liste en notant l état des cellules qu elle contient. 
* le nombre de cellules vivantes (= au voisines vivantes de la cellule à tester) sera stocké dans la cellule grâce au "self."

#### Faire évoluer la grille en respectant les 3 règles du jeu : 
   * on crée 2 listes vides pour stocker les cellules mortes et les cellules vivantes de la grille.
   * on recupère le nb de voisines vivantes de chaque cellule. 
   * on teste les 3 règles du jeu. 
   * on change l état de la cellule en fonction des règles. 
   * on affiche la nouvelle grille pour observer l évolution du jeu. 


In [ ]:
def update_board(self):
    kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])
    neighbors_count = scipy.signal.convolve2d(
        self.board, kernel, mode="same", boundary="fill", fillvalue=0
    )
    self.board = np.where(
        (self.board == 1) & ((neighbors_count < 2) | (neighbors_count > 3)),
        0,
        np.where(
            (self.board == 1) & ((neighbors_count == 2) | (neighbors_count == 3)),
            1,
            np.where((self.board == 0) & (neighbors_count == 3), 1, self.board),
        ),
    )

## 2- Le jeu en graphique: 

## 3- Analyse de l'évolution & courbes representatives: 

## 4- Le Lancement du jeu / Affichage: 
L utilisateur va lancer le jeu dans le fichier main.py 
A partir de là il aura plusieurs options : 
   * lancement en graphique avec affichage des courbes d'évolution
   * actualiser la grille manuellement ou non 
   * quitter la partie à tous moments 
Peut importe son choix il devra dans tous les cas entrer au clavier la taille de la grille souhaitée. 

Enfin, le jeu va évoluer petit à petit de façon plus ou moins autonome. 
Le joueur peut faire avancer le jeu grâce à un bouton (mode graphique) ou en tappant "q" sur son clavier (mode console).
Il est également possible de faire évoluer le jeu de manière autonome mais l évolution sera plus rapide et moins facile à analyser. 